### Import packages

In [1]:
import pandas as pd
import gensim
from gensim.utils import simple_preprocess
import os
import pyLDAvis
import pyLDAvis.gensim as gensimvis
import numpy as np
import tqdm
from tqdm import tqdm
import re
from pprint import pprint
import nltk
from nltk.corpus import stopwords
import spacy
import gensim.corpora as corpora
from pprint import pprint
from gensim.models import CoherenceModel
pd.set_option('display.max_rows', None)

C:\Users\15138\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
bbe = pd.read_csv('t_bbe.csv')

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

In [4]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [5]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [6]:
# supporting function
def compute_coherence_values_a(corpus, dictionary, k, a, b):
    lda_model = gensim.models.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [7]:
def _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus):
    corpus_sets = [corpus]
    # maximum coherence score (value)
    max_cv = 0.0
    alpha = 0.0
    beta = 0.0
    num_topics = 2

    # iterate through the corus
    for i in tqdm(range(len(corpus_sets))):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha_lst:
                # iterare through beta values
                for b in beta_lst:
                    # compute coherence value
                    cv = compute_coherence_values(corpus=corpus_sets[i], 
                                                dictionary=id2word, k=k, a=a, b=b)
                    if cv > max_cv:
                        max_cv = cv
                        alpha = a
                        beta = b
                        num_topics = k

    return alpha, beta, num_topics, max_cv

In [8]:
def _get_best_num_topics_a(topics_range, alpha_lst, beta_lst, corpus):
    corpus_sets = [corpus]
    # maximum coherence score (value)
    max_cv = 0.0
    alpha = 0.0
    beta = 0.0
    num_topics = 2

    # iterate through the corus
    for i in tqdm(range(len(corpus_sets))):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha_lst:
                # iterare through beta values
                for b in beta_lst:
                    # compute coherence value
                    cv = compute_coherence_values_a(corpus=corpus_sets[i], 
                                                dictionary=id2word, k=k, a=a, b=b)
                    if cv > max_cv:
                        max_cv = cv
                        alpha = a
                        beta = b
                        num_topics = k

    return alpha, beta, num_topics, max_cv

In [9]:
# From: https://www.machinelearningplus.com/nlp/topic-modeling-visualization-how-to-present-results-lda-models/#6.-What-is-the-Dominant-topic-and-its-percentage-contribution-in-each-document
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

# BBE

## Books

### Preprocessing

In [10]:
# Create list of book documents and put them in a dataframe
booksList = []
for book in bbe.groupby('b')['t']:
    booksList.append(' '.join(book[1].tolist()))
books = pd.DataFrame(booksList, columns =['t'])

In [11]:
# Remove punctuation
books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))

# Convert the titles to lowercase
books['t_processed'] = books['t_processed'].map(lambda x: x.lower())

#### Tokenize words and further clean-up text

data = books.t_processed.values.tolist()
data_words = list(sent_to_words(data))

#### Creating Bigram and Trigram Models

# Build the bigram and trigram models

bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5)
trigram = gensim.models.Phrases(bigram[data_words], threshold=5)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

#### Remove Stopwords, Make Bigrams and Lemmatize

#list of stopwords
with open('stop_words_lst.txt') as f:
    standard_stop_words = f.read().splitlines()
with open('bible_stop_words_lst.txt') as f:
    bible_stop_words = f.read().splitlines()
stop_words = standard_stop_words + bible_stop_words
### Call the functions above

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load("en_core_web_lg", disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

### Data transformation: Corpus and Dictionary

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

<>:2: DeprecationWarning: invalid escape sequence \.
<>:2: DeprecationWarning: invalid escape sequence \.
C:\Users\15138\AppData\Local\Temp/ipykernel_5916/1686976997.py:2: DeprecationWarning: invalid escape sequence \.
  books['t_processed'] = books['t'].map(lambda x: re.sub('[,\.!?]', '', x))


### Symmetric Average Model Selection

In [18]:
# Topics range
min_topics = 2
max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst =  [0.01, 0.5, 1.0, 2.0, 3.0]

# List of beta parameters
beta_lst = [0.01, 0.5, 1, 1.0, 2.0, 3.0]

avg_alpha = 0.0
avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 25

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics(topics_range, alpha_lst, beta_lst, corpus)
    avg_alpha += alpha
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv

avg_alpha /= num_runs
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg alpha: ", avg_alpha)
print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [37:25<00:00, 2245.67s/it]

Avg alpha:  1.1028
Avg beta:  1.82
Avg Number of topics:  6.4
Avg Optimal Coherence Value:  0.40937109728419385


### Asymmetric Average Model Parameter Selection

In [17]:
# Topics range
# Topics range
min_topics = 2
max_topics = 10
step_size = 2
topics_range = range(min_topics, max_topics, step_size)

# List of alpha parameters
alpha_lst =  ['auto']

# List of beta parameters
beta_lst = [0.01, 0.5, 1, 1.0, 2.0, 3.0]

avg_beta = 0.0
avg_num_topics = 0
avg_cv = 0.0
num_runs = 25

for run in range(num_runs):
    # Call to the method above
    alpha, beta, num_topics, max_cv = _get_best_num_topics_a(topics_range, alpha_lst, beta_lst, corpus)
    avg_beta += beta
    avg_num_topics += num_topics
    avg_cv += max_cv
    
avg_beta /= num_runs
avg_num_topics /= num_runs
avg_cv /= num_runs

print("Avg beta: ", avg_beta)
print("Avg Number of topics: ", avg_num_topics)
print("Avg Optimal Coherence Value: ", avg_cv)

100%|██████████| 1/1 [04:52<00:00, 292.46s/it]

Avg beta:  1.98
Avg Number of topics:  5.76
Avg Optimal Coherence Value:  0.40336945567563354
